<a href="https://colab.research.google.com/github/otanet/RecSys/blob/master/6_Hybrid_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hybrid Recommenders

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd /content/drive/My\ Drive/data

/content/drive/My Drive/data


In [4]:
ls

cosine_sim.csv      credits.csv   metadata_clean.csv  movies_metadata.csv
cosine_sim_map.csv  keywords.csv  movie_ids.csv       ratings_small.csv


In [0]:
!pip install scikit-surprise

In [0]:
import numpy as np
import pandas as pd

from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import PredefinedKFold

from surprise.model_selection import cross_validate

from surprise.model_selection import KFold
from surprise.model_selection import cross_validate

In [0]:
#Import or compute the cosine_sim matrix
cosine_sim = pd.read_csv('cosine_sim.csv')

In [0]:
#Import or compute the cosine sim mapping matrix
cosine_sim_map = pd.read_csv('cosine_sim_map.csv', header=None)

#Convert cosine_sim_map into a Pandas Series
cosine_sim_map = cosine_sim_map.set_index(0)
cosine_sim_map = cosine_sim_map[1]

In [29]:
#Build the SVD based Collaborative filter
from surprise import SVD, Reader, Dataset

reader = Reader()
ratings = pd.read_csv('ratings_small.csv')
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
#data.split(n_folds=5)
trainset = data.build_full_trainset()
#algo.fit(trainset)
svd = SVD()
#trainset = data.build_full_trainset()
svd.fit(trainset)
#svd.train(trainset)


# We'll use the famous SVD algorithm.
#algo = SVD()

# Run 5-fold cross-validation and print results
#cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [0]:
#Build title to ID and ID to title mappings
id_map = pd.read_csv('movie_ids.csv')
id_to_title = id_map.set_index('id')
title_to_id = id_map.set_index('title')

In [0]:
#Import or compute relevant metadata of the movies
smd = pd.read_csv('metadata_small.csv')

In [0]:
def hybrid(userId, title):
    #Extract the cosine_sim index of the movie
    idx = cosine_sim_map[title]
    
    #Extract the TMDB ID of the movie
    tmdbId = title_to_id.loc[title]['id']
    
    #Extract the movie ID internally assigned by the dataset
    movie_id = title_to_id.loc[title]['movieId']
    
    #Extract the similarity scores and their corresponding index for every movie from the cosine_sim matrix
    sim_scores = list(enumerate(cosine_sim[str(int(idx))]))
    
    #Sort the (index, score) tuples in decreasing order of similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #Select the top 25 tuples, excluding the first 
    #(as it is the similarity score of the movie with itself)
    sim_scores = sim_scores[1:26]
    
    #Store the cosine_sim indices of the top 25 movies in a list
    movie_indices = [i[0] for i in sim_scores]

    #Extract the metadata of the aforementioned movies
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    
    #Compute the predicted ratings using the SVD filter
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, id_to_title.loc[x]['movieId']).est)
    
    #Sort the movies in decreasing order of predicted rating
    movies = movies.sort_values('est', ascending=False)
    
    #Return the top 10 movies as recommendations
    return movies.head(10)

In [35]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,year,id,est
1011,The Terminator,4208.0,7.4,1984,218,3.207429
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.160871
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,3.142554
974,Aliens,3282.0,7.7,1986,679,3.094333
7705,Alice in Wonderland,8.0,5.4,1933,25694,2.938126
2834,Predator,2129.0,7.3,1987,106,2.904674
1668,Return from Witch Mountain,38.0,5.6,1978,14822,2.881617
922,The Abyss,822.0,7.1,1989,2756,2.877822
2014,Fantastic Planet,140.0,7.6,1973,16306,2.866388
8865,Star Wars: The Force Awakens,7993.0,7.5,2015,140607,2.782715


In [36]:
hybrid(2, 'Avatar')

,title,vote_count,vote_average,year,id,est
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.983453
1011,The Terminator,4208.0,7.4,1984,218,3.846616
2014,Fantastic Planet,140.0,7.6,1973,16306,3.799311
974,Aliens,3282.0,7.7,1986,679,3.711100
2834,Predator,2129.0,7.3,1987,106,3.646443
7705,Alice in Wonderland,8.0,5.4,1933,25694,3.586773
4966,Hercules in New York,63.0,3.7,1969,5227,3.578447
8865,Star Wars: The Force Awakens,7993.0,7.5,2015,140607,3.551712
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,3.536026
7088,Star Wars: The Clone Wars,434.0,5.8,2008,12180,3.490607
